# Day 11: Monkey in the Middle

[*Advent of Code 2022 day 11*](https://adventofcode.com/2022/day/11) and [*solution megathread*](https://redd.it/...)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2022/11/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2022%2F11%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')


%load_ext nb_mypy
%nb_mypy On

Version 1.0.4


In [2]:
import common


downloaded = common.refresh()
%store downloaded >downloaded

%load_ext pycodestyle_magic
%pycodestyle_on

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [3]:
from IPython.display import HTML

HTML(downloaded['part1'])

## Comments

...

In [4]:
from IPython.display import display

testdata0 = """noop
addx 3
addx -5""".splitlines()

testdata = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1""".splitlines()

inputdata = downloaded['input'].splitlines()

In [5]:
display(f'{inputdata[:10]} ... {len(inputdata)=}')

"['Monkey 0:', '  Starting items: 50, 70, 54, 83, 52, 78', '  Operation: new = old * 3', '  Test: divisible by 11', '    If true: throw to monkey 2', '    If false: throw to monkey 7', '', 'Monkey 1:', '  Starting items: 71, 52, 58, 60, 71', '  Operation: new = old * old'] ... len(inputdata)=55"

In [6]:
def take_last_int(line: str) -> int:
    return int(line.split(' ')[-1])

In [8]:
from typing import Callable


def build_operation(line: str) -> Callable[[int], int]:
    operation_str = line[line.find('new = ') +
                         len('new = '):]
    return lambda old: eval(operation_str)

In [9]:
line = 'Operation: new = old * 19'
line[line.find('new = ') + len('new = '):]

'old * 19'

In [10]:
build_operation(line)(10)

190

In [11]:
def build_divisable_test(divisor: int,
                         if_true: int,
                         if_false: int) \
                             -> Callable[[int], int]:
    return lambda sample: \
        if_true if sample % divisor == 0 \
        else if_false

In [40]:
from typing import List, Tuple, Iterable
from __future__ import annotations


class Monkey:
    def __init__(self,
                 index: int,
                 items: List[int],
                 operation: Callable[[int], int],
                 criteria: Callable[[int], int]):
        self.index = index
        self.items = items
        self.operation = operation
        self.criteria = criteria

    def __str__(self) -> str:
        return f'Monkey {self.index}: ' + \
            f'{", ".join(str(item) for item in self.items)}'

    def __repr__(self) -> str:
        return str(self) + f' ({self.operation}, {self.criteria})'

    @classmethod
    def from_lines(cls, lines: Iterable[str]) -> Monkey:
        for line in lines:
            if line.find('Monkey') != -1:
                index = int(line.split(' ')[-1][:-1])
            elif line.find('Starting items:') != -1:
                items_str = line[line.find('items: ') + len('items: '):]
                items = [int(item) for item in items_str.split(', ')]
            elif line.find('Operation:') != -1:
                operation = build_operation(line)
            elif line.find('Test:') != -1:
                divisor = take_last_int(line)
            elif line.find('If true:') != -1:
                if_true = take_last_int(line)
            elif line.find('If false:') != -1:
                if_false = take_last_int(line)
        criteria = build_divisable_test(divisor,
                                        if_true,
                                        if_false)
        return cls(index, items, operation, criteria)

    @classmethod
    def from_dataset(cls, lines: Iterable[str]) -> List[Monkey]:
        monkeys: List[Monkey] = list()
        current_lines: List[str] = list()
        for line in lines:
            if line != '':
                current_lines.append(line)
            else:
                monkeys.append(cls.from_lines(current_lines))
                current_lines = list()
        monkeys.append(cls.from_lines(current_lines))
        return monkeys

In [14]:
monkey0 = Monkey([79, 98],
                 lambda worry: worry * 19,
                 build_divisable_test(23, 2, 3))

In [34]:
monkey0_str = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3""".splitlines()

monkey0_fromstr = Monkey.from_lines(monkey0_str)

In [41]:
monkeys = Monkey.from_dataset(testdata)

In [42]:
display(monkeys)

[Monkey 0: 79, 98 (<function build_operation.<locals>.<lambda> at 0x7f6608ddc0d0>, <function build_divisable_test.<locals>.<lambda> at 0x7f6608ddd6c0>),
 Monkey 1: 54, 65, 75, 74 (<function build_operation.<locals>.<lambda> at 0x7f6608ddf910>, <function build_divisable_test.<locals>.<lambda> at 0x7f6608ddd1b0>),
 Monkey 2: 79, 60, 97 (<function build_operation.<locals>.<lambda> at 0x7f6608ddc280>, <function build_divisable_test.<locals>.<lambda> at 0x7f6608ddcc10>),
 Monkey 3: 74 (<function build_operation.<locals>.<lambda> at 0x7f6608ddcf70>, <function build_divisable_test.<locals>.<lambda> at 0x7f6608ddfac0>)]

In [44]:
def inspect_item(m: Monkey, worry: int) -> Tuple[int, int]:
    worry = m.operation(worry)
    worry //= 3
    recipient = m.criteria(worry)
    return recipient, worry

In [45]:
inspect_item(monkeys[0], 79)

(3, 500)

In [ ]:
def monkey_turn(m: Monkey, monkeys: List[Monkey]):
    for worry in items:
        recipient, worry = inspect_item(worry)
        monkeys[recipient].append(worry)
    items.clear()

In [ ]:
HTML(downloaded['part1_footer'])

## Part Two

In [ ]:
# HTML(downloaded['part2'])

In [ ]:
# HTML(downloaded['part2_footer'])